# Loading data

In [1]:
import keras

Using TensorFlow backend.


In [ ]:
!wget http://www.manythings.org/anki/fra-eng.zip

In [4]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


# Preprocessing data

In [17]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

data_path = 'data\\ben.txt'

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
print("Total number of translation pair : ", len(lines))

Total number of translation pair :  4379


In [18]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [19]:
print("input characters: ", input_characters)
print("target characters: ", target_characters)

# sample translation
for i in range(5):
  print(input_texts[i]+ "    --->    "+target_texts[i])

input characters:  {'u', 'h', ' ', 't', '%', 'I', 'T', 'Y', '€', 'V', '$', "'", '7', 'i', '?', 'y', ':', '0', '2', '5', '"', 'D', 'p', 'w', 'o', 's', 'H', 'O', 'g', ',', 'f', 'k', '1', 'n', 'v', 'd', '!', 'F', '6', '3', 'G', 'j', 'm', 'L', 'r', 'c', 'B', 'e', 'W', 'l', 'b', 'U', 'S', 'a', '8', 'P', '.', 'z', '9', 'N', 'A', 'M', 'q', 'x', 'R', '-', 'K', '4', 'C', 'J', 'E'}
target characters:  {'খ', '।', 'প', 'ু', '৮', '?', 'ঢ', 'ভ', '"', 'থ', ',', '৩', 'ধ', 'ৈ', '!', 'ো', 'শ', 'ঘ', 'ৌ', 'P', '.', 'উ', 'স', 'ত', 'ণ', '-', 'ঔ', 'চ', '১', 'ঃ', 'ৎ', 'ি', 'ঠ', 'ূ', 'ং', 'র', '৬', '০', 'া', '3', 'ন', 'অ', 'ট', '\u200c', 'ক', 'A', 'M', 'ই', 'ও', 'ঐ', 'ঞ', 'ব', 'ী', 'য়', 'জ', 'ষ', '\u200d', 'ঝ', '\t', '1', 'য', 'ড়', '৭', 'ছ', 'ে', 'ঁ', 'ল', 'U', 'ৃ', ' ', '%', '\n', '€', '$', "'", 'ম', ':', '৫', '৯', '়', 'হ', 'ঊ', 'গ', 'এ', 'ঈ', 'দ', 'ড', 'ফ', '্', 'ঙ', '২', '৪', 'আ'}
Go.    --->    	যাও।

Go.    --->    	যান।

Go.    --->    	যা।

Run!    --->    	পালাও!

Run!    --->    	পালান!



In [20]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 4378
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 96
Max sequence length for outputs: 95


### Assign token index

In [21]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

### Prepare one-hot encoder data representation : Shape (batch, sentence_len, encode_dim)

In [22]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("encoder input data shape: ", encoder_input_data.shape)
print("decoder input data shape: ", decoder_input_data.shape)
print("decoder target data shape: ", decoder_target_data.shape)

encoder input data shape:  (4378, 96, 71)
decoder input data shape:  (4378, 95, 93)
decoder target data shape:  (4378, 95, 93)


### decoder input :   HELLO    --->   decoder target:  ELLO

In [23]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Build the Sequence to Sequence model

In [24]:
# Define input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
# LSTM as encoder
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# we only need 'encoder_outputs'
encoder_states = [state_h, state_c]


# set up the decoder, using `encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# we set up decoder to return full output sequences, and to return internal
# states as well. we don't use the return states in training model, but
# we will use them in referance
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# define a model that will turn 'encoder_input_data' & 'decoder_input_data'
# into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 93)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 335872      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  358400      input_6[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [26]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2
         )


Train on 3502 samples, validate on 876 samples
Epoch 1/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.7031 - val_loss: 1.2541
Epoch 2/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.6381 - val_loss: 1.1626
Epoch 3/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.5664 - val_loss: 1.0366
Epoch 4/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.5121 - val_loss: 0.9844
Epoch 5/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.4746 - val_loss: 0.9499
Epoch 6/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.4484 - val_loss: 0.9090
Epoch 7/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.4306 - val_loss: 0.9012
Epoch 8/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.4128 - val_loss: 0.9118
Epoch 9/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.3989 - val_loss: 0.8614
Epoch 10/100
3502

3502/3502 [==============================] - 10s 3ms/step - loss: 0.1032 - val_loss: 1.1931
Epoch 79/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.1020 - val_loss: 1.2132
Epoch 80/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.1013 - val_loss: 1.2062
Epoch 81/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.1002 - val_loss: 1.2194
Epoch 82/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.0984 - val_loss: 1.2108
Epoch 83/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.0981 - val_loss: 1.2260
Epoch 84/100
3502/3502 [==============================] - 10s 3ms/step - loss: 0.0980 - val_loss: 1.2621
Epoch 85/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.0968 - val_loss: 1.2599
Epoch 86/100
3502/3502 [==============================] - 11s 3ms/step - loss: 0.0963 - val_loss: 1.2546
Epoch 87/100
3502/3502 [==============================] - 11s 3ms/st

OSError: Unable to create file (unable to open file: name = 'saved_model\s2s.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [27]:
model.save('saved_models\\s2s.h5')

c:\users\bikas\anaconda3\lib\site-packages\keras\engine\topology.py:2368: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


# Inferance with this model

In [33]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs]+decoder_state_inputs,
    [decoder_outputs]+decoder_states
)

### One-hot output to char

In [34]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### Decode sequence

In [35]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [36]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Go.
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Go.
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Run!
Decoded sentence: আমাকে একটা দেখান।

-
Input sentence: Run!
Decoded sentence: আমাকে একটা দেখান।

-
Input sentence: Who?
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Fire!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Help!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Help!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Stop!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Stop!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Stop!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Hello!
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: I see.
Decoded sentence: আমি একটা চিন্তি পর্বাস নি।

-
Input sentence: I try.
Decoded sentence: আমি একটা চিন্তি নি।

-
Input sentence: Smile.
Decoded sentence: টম চিৎকার করছে।

-
Input sentence: Smile.
Decoded sentence: